In [1]:
import pandas as pd
from tqdm import tqdm
import ollama
import requests
import json
import time
import os

In [ ]:
START_DATE = "2018-01-01"
# END_DATE = "2021-12-31"
END_DATE = "2020-01-01"

TICKERS = ["AAPL", "AMZN", "GOOGL", "MSFT", "NVDA", "TSLA"]

In [ ]:
# def query_ollama(prompt):
#     response = ollama.chat(
#         model=MODEL,
#         messages=[
#             {
#                 "role": "user",
#                 "content": prompt,
#             },
#         ],
#     )
#     return response["message"]["content"]

In [ ]:
# def extract_factors(stock_ticker, news_content, k=5):
#     prompt = f"""
#     Please extract the top {k} factors that may affect the stock price of {stock_ticker} from the following news.
    
#     {news_content}
#     """
#     return query_ollama(prompt)

In [ ]:
def was_market_open(date):
    df = pd.read_csv("../data/were-markets-open.csv")

    # Reduce to rows where 'was_open' is True
    df = df[df["was_open"]]

    return date in df["date"].values

In [6]:
# Generate a list of dates between START_DATE and END_DATE as strings
dates = pd.date_range(start=START_DATE, end=END_DATE).strftime("%Y-%m-%d")
dates

Index(['2018-01-01', '2018-01-02', '2018-01-03', '2018-01-04', '2018-01-05',
       '2018-01-06', '2018-01-07', '2018-01-08', '2018-01-09', '2018-01-10',
       ...
       '2019-12-23', '2019-12-24', '2019-12-25', '2019-12-26', '2019-12-27',
       '2019-12-28', '2019-12-29', '2019-12-30', '2019-12-31', '2020-01-01'],
      dtype='object', length=731)

In [10]:
# Modify the loop that executes trades based on the model's response
for ticker in TICKERS:
    print(f"Processing {ticker}...")
    for date in tqdm(dates):
        # Skip if the factors file already exists
        if os.path.exists(f"factors/{ticker}-{date}.md"):
            continue

        # Ensure that the market was open on the given date
        if not was_market_open(date):
            continue

        news_data = ""
        try:
            data = requests.get(f"http://localhost:8000/{ticker}/{date}")
            data = data.json()
        except:
            # Formatting issue with the dataset
            # Log offending ticker/date to a .txt file
            with open("error.log", "a") as f:
                f.write(f"{ticker}, {date}\n")

            # TODO: Investigate why some JSON is malformed
            # example: AAPL, 2019-04-11
            continue

        for news in data["news"]:
            news_data += f"""
                ### {news["title"]}
                
                    {news["summary"]}
                """

        factors = extract_factors(
            ticker, news_data
        )
        
        print(factors)
        # with open(f"factors/{ticker}-{date}.md", "w") as f:
        #     f.write(factors)

Processing AAPL...


  0%|          | 2/731 [00:17<1:49:20,  9.00s/it]

The article discusses various aspects related to Apple Inc., including its iPhone market share, potential acquisition of Netflix, and battery life issues with older iPhone models.

Some key points mentioned in the article are:

*   Apple faces challenges in growing its share of the high-end smartphone market.
*   Analysts speculate that there's a 40% chance Apple will buy Netflix due to recent tax reforms allowing it to comfortably finance a big acquisition.
*   A survey of 400 iPhone buyers has boosted Piper Jaffray analysts' expectations for the mix of iPhones to be sold in fiscal 2018.
*   Apple offers $29 battery replacement for iPhone 6 and later devices to address concerns over slowing down older iPhones.

Overall, the article highlights various factors affecting Apple's performance and potential opportunities.


  0%|          | 2/731 [00:29<3:01:58, 14.98s/it]


KeyboardInterrupt: 